## 📌 **Cell 1 — Load Gemini API Key**

- This cell loads your **Google Gemini API key** using Kaggle’s `UserSecretsClient`.
- If the key exists, it stores it in the environment variable `GOOGLE_API_KEY`.
- If missing, it prints an error message.

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print("❌ API Key error:", e)

✅ Gemini API key setup complete.


## 📌 **Cell 2 — Import ADK Components**

This cell imports all required **Google ADK components**:

* `Agent`, `SequentialAgent` for agent behavior
* `Gemini` model wrapper
* `Runner` executor
* Built-in tools like `google_search`
* In-memory session manager
* Other utilities like `datetime`, `uuid`, `requests`

This sets up the foundational building blocks for the GeoAgent system.


In [2]:
from google.adk.agents import Agent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.tools import FunctionTool, google_search
from google.adk.sessions import InMemorySessionService
from google.genai import types
from datetime import datetime
import requests
import uuid

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


## 📌 **Cell 3 — Define Async Session Runner**

Defines an asynchronous helper function `run_session()` that:

* Creates or retrieves a session
* Accepts one or more user queries
* Converts each query into ADK Content format
* Streams back the agent’s response in real time

This is the main execution engine used to test the agent system.

In [3]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{runner_instance.app_name} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


## 📌 **Cell 4 — API Retry Policy**

Configures a retry mechanism for Gemini API calls:

* Up to 5 retry attempts
* Exponential backoff
* Retries for common transient HTTP errors (429, 500, 503, 504)

Improves agent robustness.

In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

print("✅ Retry config set.")

✅ Retry config set.


## 📌 **Cell 5 — Earthquake Data Tool**

Defines a custom Python function:

* Calls USGS Earthquake API
* Fetches major (M ≥ 7.5) earthquakes from the last X days
* Returns a short textual list
* Wrapped into a `FunctionTool` so agents can call it

This is the dedicated seismic-data tool for the earthquake researcher agent.

In [5]:
def get_earthquake_data(location: str = "global", days: int = 183) -> str:
    """
    Fetches recent earthquake data from USGS.
    """
    try:
        url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
        params = {
            "format": "geojson",
            "starttime": f"{days}daysago",
            "minmagnitude": 7.5,
            "limit": 5
        }
        response = requests.get(url, params=params)
        data = response.json()

        earthquakes = []
        for eq in data.get("features", [])[:5]:
            mag = eq["properties"]["mag"]
            place = eq["properties"]["place"]
            earthquakes.append(f"M{mag} - {place}")

        return "\n".join(earthquakes) if earthquakes else "No significant earthquakes found."
    except Exception as e:
        return f"Could not fetch earthquake data: {e}"


earthquake_tool = FunctionTool(get_earthquake_data)

print("✅ Earthquake tool loaded.")

✅ Earthquake tool loaded.


## 📌 **Cell 6 — Session Service Initialization**

Creates constants:

* `APP_NAME`
* `USER_ID`

Initializes `InMemorySessionService`, enabling agents to maintain memory across messages within a session.

In [6]:
# Create a session setup variables
APP_NAME = "GeoAgent "  # Application
USER_ID = "default"  # User

# Create the session service
session_service = InMemorySessionService()

print("✅ Session service added (InMemorySessionService).")

✅ Session service added (InMemorySessionService).


## 📌 **Cell 7 — Geoscience Research Agent**

Creates the **GeoscienceResearcher** agent:

* Uses Gemini Flash Lite
* Searches recent geoscience publications
* Outputs:

  * 5 new research findings
  * 3 GeoInformatics advances
* Strict formatting rules
* Uses Google search tool

First step of the research pipeline.

In [7]:
geo_researcher = Agent(
    name="GeoscienceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
Research the latest Geoscience and GeoInformatics breakthroughs.
STRICT & ONLY Provide:
- 5 new findings in Geoscience (from research papers)
- 3 advances in GeoInformatics (GIS/Remote Sensing)
Max 300 words.

Format the output with:
• STRICT start with : 🌎 **Here's a briefing on recent Geoscience and GeoInformatics breakthroughs:**:
• STRICT & ONLY Format: 
        **Geosciences new findings from papers** :
        **GeoInformatics breakthroughs** :
""",
    tools=[google_search],  
    output_key="geo_research",
)

print("✅ geo_researcher created.")

✅ geo_researcher created.


## 📌 **Cell 8 — Earthquake Research Agent**

Creates the **EarthquakesResearcher** agent:

* Only calls the `earthquake_tool`
* Produces disaster updates ONLY (no extra info)
* Strict formatting:

  * Title: “🌋 Recent Mega Earthquakes:”
  * One line per event

Second step in the research workflow.

In [8]:
earthquakes_researcher = Agent(
    name="EarthquakesResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
Your SOLE and STRICT task is to produce a disaster update.
1. Call the 'earthquake_tool' to fetch recent global earthquakes.
2. Return ONLY a list of earthquakes, strictly adhering to the specified format below.

STRICT RULES:
- IGNORE ALL PREVIOUS AGENT OUTPUTS IN THIS WORKFLOW (e.g., ignore GeoscienceResearcher).
- DO NOT generate any content that is not a direct result of the earthquake_tool, or the required title/formatting.
- DO NOT write any geoscience, geology, geoinformatics, climate, or news briefing.
- DO NOT repeat or summarize content from any other agent.
- Output must be <150 words.

Format the output with:
• The output MUST start with: 🌋 **Recent Mega Earthquakes**:
• One line per earthquake.
• Format for each line: **Location (Magnitude)** - Short geological interpretation
""",
    tools=[earthquake_tool],\
    output_key="earthquakes_researcher",
)

print("✅ earthquakes_researcher created.")

✅ earthquakes_researcher created.


## 📌 **Cell 9 — Summary Agent**

Creates a **SummaryAgent** that:

* Reads outputs from the two previous agents
* Produces a 200-word executive briefing
* Includes:

  * Core insights
  * Connections & implications
  * Final summarized section

This is the final aggregation layer of the research workflow.

In [9]:
summary_agent = Agent(
    name="SummaryAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
Combine:

**Geoscience / GeoInfo Research**
{geo_research}

**Earthquakes Latest**
{earthquakes_researcher}

Write a 200-word executive briefing summarizing:
- Core insights
- Connections between trends
- Significant implications

STRICT RULES:
- Do NOT output any greetings, personal information (like the user's name), or conversational remarks.
- The output MUST start directly with the formatted briefing.
- Format of the output: 📄**Summary: Geoscience, GeoInformatics and latest earthquakes **
• Format: 
    **Core Insights** :
    **Connections & Implications** :
    **Summary** :
""",
    output_key="executive_summary",
)

print("✅ summary_agent created.")

✅ summary_agent created.


## 📌 **Cell 10 — Conversational Agent**

Defines a lightweight agent to answer:

* Simple personal/context questions
* Using ONLY session history
* No tools, no research allowed
* Remains silent for complex queries

This prevents the model from giving hallucinated personal info.

In [10]:
conversational_agent = Agent(
    name="ChatAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="You are a friendly and concise conversational agent. Your sole purpose is to answer simple questions about the user's name, work, or other context **strictly** retrieved from the session history. You MUST NOT use any tools or attempt complex research. If the query is complex or involves research, you must **remain silent** and let the coordinator route the request.",
)
print("✅ conversational_agent created.")

✅ conversational_agent created.


## 📌 **Cell 11 — Sequential Research Workflow**

Defines a **SequentialAgent**:

`GeoscienceResearcher → EarthquakesResearcher → SummaryAgent`

This ensures the agents run in order and feed each other’s outputs.

In [11]:
research_workflow_agent = SequentialAgent(
    name="ComplexResearchWorkflow",
    sub_agents=[geo_researcher, earthquakes_researcher, summary_agent],
)
print("✅ ComplexResearch workflow created (SequentialAgent).")

✅ ComplexResearch workflow created (SequentialAgent).


## 📌 **Cell 12 — GeoAgent Coordinator (Router)**

Creates the routing agent:

* Inspects user intent
* If message involves research → route to **ComplexResearchWorkflow**
* If simple conversational message → route to **ChatAgent**

Serves as the brain of the entire GeoAgent architecture.

In [12]:
root_agent = Agent(
    name="GeoAgentCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    STRICT:
    You are the GeoAgent system coordinator. Your task is to analyze the user's intent and route the message to the correct sub-agent.
    - If the user asks a conversational question include a 'briefing', 'research', 'summary', or mentions 'geoscience/geoinfo/earthquakes', ONLY delegate to the 'ComplexResearchWorkflow' WITHOUT ADD ANY PERSONAL INFO.
    - If the user asks a simple conversational question (greeting, name, job, etc.), ONLY delegate to the 'ChatAgent'.
    """,
    # The Router delegates between the two high-level paths
    sub_agents=[conversational_agent, research_workflow_agent]
)
print("✅ GeoAgentCoordinator (Router) created.")

✅ GeoAgentCoordinator (Router) created.


## 📌 **Cell 13 — Runner Setup**

Creates a **Runner** instance:

* Connects the coordinator agent
* Attaches the session service
* Associates with the app name

This is what actually executes agent logic.

In [13]:
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service
)
print("✅ Runner configured with the Router Agent.")

✅ Runner configured with the Router Agent.


## 📌 **Cell 14 — Run Example User Session**

Generates a random session ID and runs 3 user queries:

1. Geoscience/GeoInfo daily briefing
2. User name request
3. User job request

This tests routing:

* Query 1 → goes to research workflow
* Queries 2–3 → go to conversation agent

If any error occurs, it prints “FULL ERROR”.

In [14]:
SESSION = str(uuid.uuid4())

try:
    response = await run_session(
    runner,
    [
        "Hi, I am Ahmed, geoscience publisher! Write to me the daily briefing on Geoscience / GeoInfo, and latest earthquakes?",
        "Hello! What is my name?",
        "Hi! What is my work?", 
    ],
    SESSION,
)
except Exception as e:
    print("\n❌ FULL ERROR:", e)


 ### Session: b78e0001-7570-4077-a733-9386310c100c

User > Hi, I am Ahmed, geoscience publisher! Write to me the daily briefing on Geoscience / GeoInfo, and latest earthquakes?


GeoAgent  >  🌎 **Here's a briefing on recent Geoscience and GeoInformatics breakthroughs:**

**Geosciences new findings from papers**:
*   New research reveals that the Earth's inner core rotation may have slowed down or even reversed relative to the surface in recent years, potentially influencing Earth's magnetic field and day length.
*   A study published in *Nature Geoscience* suggests that the deep carbon cycle plays a more significant role in regulating Earth's long-term climate than previously understood, influencing volcanic activity and atmospheric CO2 levels over geological timescales.
*   Scientists have identified novel microbial communities thriving in deep-sea hydrothermal vents, demonstrating remarkable metabolic adaptations to extreme conditions and offering insights into the origins of life.
*   Analysis of ancient lake sediments has uncovered evidence of past megadroughts in southwestern North America occurring with surprising frequency, raising concerns about future 

GeoAgent  >  🌋 **Recent Mega Earthquakes**:
**Drake Passage (7.6)** - A significant seismic event occurred in the Drake Passage, indicating tectonic activity in the region south of South America.
**145 km E of Petropavlovsk-Kamchatsky, Russia (7.8)** - A major earthquake near the Kamchatka Peninsula, a volcanically active area known for frequent seismic events due to its location on the Pacific Ring of Fire.
**2025 Southern Drake Passage Earthquake (7.5)** - This earthquake in the southern Drake Passage suggests continued tectonic stress and adjustments in the Scotia Plate boundary.
**2025 Kamchatka Peninsula, Russia Earthquake (8.8)** - An extremely powerful earthquake in the Kamchatka region, highlighting the immense tectonic forces at play along the Kuril-Kamchatka subduction zone.
GeoAgent  >  📄**Summary: Geoscience, GeoInformatics and latest earthquakes **
    **Core Insights** :
    Recent geoscience research points to shifts in Earth's inner core rotation, potentially impacting 